# Numero de Nusselt

In [1]:
import pandas as pd
import matplotlib 
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.random import check_random_state
from sklearn.metrics import mean_squared_error, r2_score
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import export_graphviz
import graphviz
import pandas as pd


In [2]:
# cargando datos

# importacion de DF
file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_n_25.txt'
df_n_25 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_n_53.txt'
df_n_53 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_n_74.txt'
df_n_74 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_n_102.txt'
df_n_102 = pd.read_csv(file_path, delimiter=',')

In [3]:
# definiendo conjuntos de train y test

# definiendo conjunto de train
df_n_train = pd.concat([df_n_25, df_n_74], ignore_index=True)

# separando entre x e y
y_train = df_n_train.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','prandtl','colIndex'])
X_train = df_n_train.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','nusselt'])


# definiendo conjunto de test

y_test = df_n_53.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','prandtl','colIndex'])
X_test = df_n_53.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','nusselt'])

### Entrenar modelo original

In [5]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=30, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    32.83      4.89184e+31       31          11.7311            11.78     20.90m
   1    17.97      6.65547e+15       31          9.48876          9.30918     14.81m
   2    26.49      4.08048e+09       27          6.99431          6.97922     24.02m
   3    30.02      1.92981e+13       27          7.00878          6.84901     16.49m
   4    29.69       1.2269e+11       27          6.93223          7.53787     12.27m
   5    32.47       4.5187e+07       25          6.67603          6.71544     12.32m
   6    35.73       1.3954e+09       67          3.30332          3.29504     11.02m
   7    38.75      1.40532e+10       57          3.24255          3.28773     11.20m
   8    38.52      7.44973e+07       57          3.23477          3.35773  

In [6]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 11.00072279485055
R^2: 0.9773198439795644


### Entrenar agregando potencias

In [8]:
# creando funciones auxiliares

# S ^(-0.2) 
def pot1(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, np.power(x1, -0.2), 0)
    return result

pot1_fn = make_function(function=pot1, 
                        name='pot1', 
                        arity=1)

# 0.5 Re ^ (0.64)
def pot2(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, 0.5*np.power(x1, 0.64), 0)
    return result

pot2_fn = make_function(function=pot2, 
                        name='pot2', 
                        arity=1)



In [9]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=25, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn, pot2_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    17.31      1.28917e+18       31          3.44956          3.49005     22.60m
   1    14.55          38461.8       19          2.40209           2.4134     18.03m
   2    20.72          83461.2       13          2.21551          2.21593     28.03m
   3    27.49          38385.9       30          1.83343          1.76745     29.17m
   4    22.86          4679.08       35          1.81892          1.89808     23.20m
   5    20.79          551.603       25          1.67295          1.73142     26.10m
   6    24.68          22297.9       33          1.66798          1.75921     29.30m
   7    20.71          7257.64       28          1.66958          1.76174     20.94m
   8    16.98          869.651       33          1.66608          1.70798  

In [10]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 4.909670601450528
R^2: 0.9898777473692939
